In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing


In [6]:
df = pd.read_csv("C:\\Users\\kyawt\\OneDrive\\Desktop\\eastern university data science\\ML2\\6\\mushrooms\\agaricus-lepiota.data")


In [7]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

one_hot_encoder = OneHotEncoder()
label_encoder = LabelEncoder()

def preprocess(df):
  feature_columns = df.loc[:, df.columns != 'e.1'].columns
  encoded_features = one_hot_encoder.fit_transform(df[feature_columns])
  encoded_label = label_encoder.fit_transform(df["e.1"])
  return encoded_features, encoded_label

encoded_features, encoded_labels = preprocess(df.copy())

In [8]:
missingIdx = []
fullIdx = []
for idx, row in df.iterrows():
    if any('?' in cell for cell in row):
        missingIdx.append(idx)
    else:
        fullIdx.append(idx)

full_encoded_feature = encoded_features[fullIdx]
full_encoded_labels = encoded_labels[fullIdx]

missing_encoded_feature = encoded_features[missingIdx]
missing_encoded_labels = encoded_labels[missingIdx]

In [9]:
# train KNN model on df_full_encoded
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Create train & test split
X = full_encoded_feature
y = full_encoded_labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training
k = 5  # 5 neighbors (default)
knn_classifier = KNeighborsClassifier(n_neighbors=k)
knn_classifier.fit(X_train, y_train)

# Evaluation
y_pred = knn_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 1.00


In [10]:
predicted_labels = knn_classifier.predict(missing_encoded_feature)
missing_values = label_encoder.inverse_transform(predicted_labels)

In [11]:
unique_values, counts = np.unique(missing_values, return_counts=True)
for value, count in zip(unique_values, counts):
    print(f"Value: {value}, Count: {count}")

Value: b, Count: 1888
Value: c, Count: 64
Value: e, Count: 528


In [12]:
missing_values[0]

'b'

In [15]:
i = 0
df_cpy = df.copy()
for idx, row in df.iterrows():
    if row['e.1'] == '?':
        df_cpy.at[idx, 'e.1'] = missing_values[i]
        i += 1


In [14]:
df_cpy.head()

,p,x,s,n,t,p.1,f,c,n.1,k,e,e.1,s.1,s.2,w,w.1,p.2,w.2,o,p.3,k.1,s.3,u,"(8122, e.1)"
0,e,x,s,y,t,a,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n,n,g,e
1,e,b,s,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m,e
2,p,x,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k,s,u,e
3,e,x,s,g,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g,e
4,e,x,y,y,t,a,f,c,b,n,e,c,s,s,w,w,p,w,o,p,k,n,g,e


In [16]:
if (df_cpy == '?').any().any():
    print("There are still missing values in the dataframe.")
else:
    print("All missing values in the dataframe have been imputed.")


All missing values in the dataframe have been imputed.


In [17]:
df_cpy


,p,x,s,n,t,p.1,f,c,n.1,k,e,e.1,s.1,s.2,w,w.1,p.2,w.2,o,p.3,k.1,s.3,u
0,e,x,s,y,t,a,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n,n,g
1,e,b,s,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m
2,p,x,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k,s,u
3,e,x,s,g,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g
4,e,x,y,y,t,a,f,c,b,n,e,c,s,s,w,w,p,w,o,p,k,n,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8118,e,k,s,n,f,n,a,c,b,y,e,e,s,s,o,o,p,o,o,p,b,c,l
8119,e,x,s,n,f,n,a,c,b,y,e,e,s,s,o,o,p,n,o,p,b,v,l
8120,e,f,s,n,f,n,a,c,b,n,e,b,s,s,o,o,p,o,o,p,b,c,l
8121,p,k,y,n,f,y,f,c,n,b,t,b,s,k,w,w,p,w,o,e,w,v,l
